# Исследование надежности заемщиков


## Откроем таблицу и изучим общую информацию о данных

Импортируем библиотеку pandas. Считаем данные из csv-файла в датафрейм и сохраним в переменную `data`. Путь к файлу:

`/datasets/data.csv`

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

Выведем первые 20 строчек датафрейма `data` на экран.

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Выведем основную информацию о датафрейме с помощью метода `info()`.

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

Выведем количество пропущенных значений для каждого столбца.

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [5]:
for t in data['income_type'].unique()
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`.

In [6]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получаются аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть.

Выведем перечень уникальных значений столбца `children`.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски заполнены. Проверим себя и ещё раз выведем количество пропущенных значений для каждого столбца с помощью двух методов.

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру. Проверим остальные столбцы.

In [14]:
data['education'] = data['education'].str.lower()

Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используем собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используем собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучим данные в столбце `purpose` и определим, какие подстроки помогут нам правильно определить категорию.**

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуем данные и ответим на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:
print(data['debt'].value_counts())
# всего 1 732 клиента из выборки (8,8%) допускали просрочки

0    19599
1     1732
Name: debt, dtype: int64


In [23]:
print(data['children'].value_counts())
# получаем распределение клиентов в зависимости от количества детей. Клиентов, имеющих 4 и 5 детей, относительно мало - выборка по ним будет не репрезентативна.

0    14091
1     4808
2     2052
3      330
4       41
5        9
Name: children, dtype: int64


In [24]:
print(data['children'].value_counts())
# получаем распределение "неплательщиков" по количеству детей. Цифры не показательны, так как клиенты по количеству детей распределены неравномерно

0    14091
1     4808
2     2052
3      330
4       41
5        9
Name: children, dtype: int64


In [25]:
data_grouped = data.groupby(['children']).agg({'debt':'sum'}) # получаем количество "неплательщиков" в каждой категории клиентов (в зависимости от количества детей) другим способом
print(data_grouped)
# цифры не показательны, так как клиенты по количеству детей распределены неравномерно

          debt
children      
0         1063
1          444
2          194
3           27
4            4
5            0


In [26]:
data_grouped_avg = data.groupby('children')['debt'].mean()
print(data_grouped_avg)
# определяем, какова доля неплательщиков в каждой категории клиентов

children
0    0.075438
1    0.092346
2    0.094542
3    0.081818
4    0.097561
5    0.000000
Name: debt, dtype: float64


In [27]:
print(f'Доля неплательщиков среди бездетных составляет {data_grouped_avg[0]:.2%}')
print(f'Доля неплательщиков среди имеющих 1 ребенка составляет {data_grouped_avg[1]:.2%}')
print(f'Доля неплательщиков среди имеющих 2 детей составляет {data_grouped_avg[2]:.2%}')
print(f'Доля неплательщиков среди имеющих 3 детей составляет {data_grouped_avg[3]:.2%}')
print(f'Доля неплательщиков среди имеющих 4 детей составляет {data_grouped_avg[4]:.2%}')
print(f'Доля неплательщиков среди имеющих 5 детей составляет {data_grouped_avg[5]:.2%}')
# отображаем данные в более читабельном формате

Доля неплательщиков среди бездетных составляет 7.54%
Доля неплательщиков среди имеющих 1 ребенка составляет 9.23%
Доля неплательщиков среди имеющих 2 детей составляет 9.45%
Доля неплательщиков среди имеющих 3 детей составляет 8.18%
Доля неплательщиков среди имеющих 4 детей составляет 9.76%
Доля неплательщиков среди имеющих 5 детей составляет 0.00%


In [42]:
debtors = data[data['debt'] == 1]
percentage_0 = debtors[debtors['children'] == 0].shape[0] / data[data['children'] == 0].shape[0]
percentage_1 = debtors[debtors['children'] != 0].shape[0] / data[data['children'] != 0].shape[0]
print(f'Доля неплательщиков среди бездетных составляет {percentage_0:.2%}')
print(f'Доля неплательщиков среди имеющих детей (любое количество) составляет {percentage_1:.2%}')
# сравниваем долю "неплательщиков" среди бездетных клиентов и имеющих хотя бы одного ребенка

Доля неплательщиков среди бездетных составляет 7.54%
Доля неплательщиков среди имеющих детей (любое количество) составляет 9.24%


In [29]:
data_pivot_children = data.pivot_table(index=['children'], values='debt', aggfunc='mean')
data_pivot_children['debt'] = data_pivot_children['debt'] * 100
data_pivot_children['debt'] = data_pivot_children['debt'].round(2)
data_pivot_children
# определяем другим способом (с помощью сводной таблицы), какова доля неплательщиков в каждой категории клиентов

,debt
children,
0,7.54
1,9.23
2,9.45
3,8.18
4,9.76
5,0.00


**Вывод:** 
Результаты анализа показывают, что определенная зависимость между фактом наличия детей и возвратом кредита в срок имеется: доля имевших задолженность по кредиту среди бездетных немного ниже. При этом прямой зависимости между количеством детей и возвратом кредита в срок не обнаружено (доля неплательщиков среди имеющих 3 детей ниже, чем среди имеющих 1 или 2).
Таким образом, возможно сделать вывод, что более надежным при прочих равных является заемщик без детей.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [30]:
print(data['family_status'].value_counts())
# получаем распределение клиентов по семейному положению

женат / замужем          12261
гражданский брак          4134
Не женат / не замужем     2796
в разводе                 1189
вдовец / вдова             951
Name: family_status, dtype: int64


In [31]:
print(debtors['family_status'].value_counts())
# получаем распределение "неплательщиков" по семейному положению

женат / замужем          927
гражданский брак         385
Не женат / не замужем    273
в разводе                 84
вдовец / вдова            63
Name: family_status, dtype: int64


In [32]:
data_grouped_avg_1 = data.groupby('family_status')['debt'].mean()
print(data_grouped_avg_1)
# определяем, какова доля неплательщиков в каждой категории клиентов

family_status
Не женат / не замужем    0.097639
в разводе                0.070648
вдовец / вдова           0.066246
гражданский брак         0.093130
женат / замужем          0.075606
Name: debt, dtype: float64


In [33]:
print(f'Доля неплательщиков среди не состоящих в браке составляет {data_grouped_avg_1[0]:.2%}')
print(f'Доля неплательщиков среди разведенных составляет {data_grouped_avg_1[1]:.2%}')
print(f'Доля неплательщиков среди овдовевших составляет {data_grouped_avg_1[2]:.2%}')
print(f'Доля неплательщиков среди состоящих в гражданском браке составляет {data_grouped_avg_1[3]:.2%}')
print(f'Доля неплательщиков среди состояющих в браке составляет {data_grouped_avg_1[4]:.2%}')
# отображаем данные в более читабельном формате

Доля неплательщиков среди не состоящих в браке составляет 9.76%
Доля неплательщиков среди разведенных составляет 7.06%
Доля неплательщиков среди овдовевших составляет 6.62%
Доля неплательщиков среди состоящих в гражданском браке составляет 9.31%
Доля неплательщиков среди состояющих в браке составляет 7.56%


In [34]:
data_pivot_status = data.pivot_table(index=['family_status'], values='debt', aggfunc='mean')
data_pivot_status['debt'] = data_pivot_status['debt'] * 100
data_pivot_status['debt'] = data_pivot_status['debt'].round(2)
data_pivot_status
# определяем другим способом (с помощью сводной таблицы), какова доля неплательщиков в каждой категории клиентов

,debt
family_status,
Не женат / не замужем,9.76
в разводе,7.06
вдовец / вдова,6.62
гражданский брак,9.31
женат / замужем,7.56


**Вывод:** 
Результаты анализа показывают, что определенная зависимость между семейным положением и возвратом кредита в срок имеется: доля имевших задолженность по кредиту среди не состоящих в браке и состоящих в гражданском браке выше, чем в других категориях.
Таким образом, возможно сделать вывод, что более надежным при прочих равных является заемщик, состоящий либо состоявший в браке (при этом выборка по разведенным и овдовевшим не репрезентативна).

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [35]:
print(data['total_income_category'].value_counts())
# получаем распределение клиентов по доходу, применяя ранее созданную категоризацию. Клиентов, зарабатывающих менее 30 тыс. и более 1 млн, очень мало.

C    15921
B     5014
D      349
A       25
E       22
Name: total_income_category, dtype: int64


In [36]:
data_grouped_avg_2 = data.groupby('total_income_category')['debt'].mean()
print(data_grouped_avg_2)
# определяем, какова доля неплательщиков в каждой категории клиентов

total_income_category
A    0.080000
B    0.070602
C    0.084982
D    0.060172
E    0.090909
Name: debt, dtype: float64


In [37]:
print(f'Доля неплательщиков среди зарабатывающих от 1 млн составляет {data_grouped_avg_2[0]:.2%}')
print(f'Доля неплательщиков среди зарабатывающих от 200 тыс. до 1 млн составляет {data_grouped_avg_2[1]:.2%}')
print(f'Доля неплательщиков среди зарабатывающих от 50 тыс. до 200 тыс. составляет {data_grouped_avg_2[2]:.2%}')
print(f'Доля неплательщиков среди зарабатывающих от 30 тыс. до 50 тыс. составляет {data_grouped_avg_2[3]:.2%}')
print(f'Доля неплательщиков среди зарабатывающих до 30 тыс. составляет {data_grouped_avg_2[4]:.2%}')
# отображаем данные в более читабельном формате

Доля неплательщиков среди зарабатывающих от 1 млн составляет 8.00%
Доля неплательщиков среди зарабатывающих от 200 тыс. до 1 млн составляет 7.06%
Доля неплательщиков среди зарабатывающих от 50 тыс. до 200 тыс. составляет 8.50%
Доля неплательщиков среди зарабатывающих от 30 тыс. до 50 тыс. составляет 6.02%
Доля неплательщиков среди зарабатывающих до 30 тыс. составляет 9.09%


In [38]:
data_pivot_income = data.pivot_table(index=['total_income_category'], values='debt', aggfunc='mean')
data_pivot_income['debt'] = data_pivot_income['debt'] * 100
data_pivot_income['debt'] = data_pivot_income['debt'].round(2)
data_pivot_income
# определяем другим способом (с помощью сводной таблицы), какова доля неплательщиков в каждой категории клиентов

,debt
total_income_category,
A,8.00
B,7.06
C,8.50
D,6.02
E,9.09


**Вывод:** 
Клиентов, зарабатывающих менее 30 тыс. и более 1 млн руб. относительно мало, в связи с чем выборка по ним не может быть признана репрезентативной в полной мере. Доля неплательщиков в категории клиентов с доходом от 30 тыс. до 50 тыс. ниже, чем в других категориях, однако таких клиентов также относительно немного. При этом среди двух основных категорий клиентов по доходам (B и C) меньше доля неплательщиков все-таки среди зарабатывающих больше. В целом возможно провести более детальный анализ на предмет определение влияния иных дополнительных факторов на платежную дисциплину клиентов.

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [39]:
print(data['purpose_category'].value_counts())
# получаем распределение клиентов по целям кредитования. Видим, что клиентов, получивших кредит на проведение свадьбы, относительно немного (10,8%).
# Впрочем, остальные категории также существенно меньше, чем количество заемщиков, получивших ссуду на операции с недвижимостью (50,4%).

операции с недвижимостью    10751
операции с автомобилем       4279
получение образования        3988
проведение свадьбы           2313
Name: purpose_category, dtype: int64


In [40]:
data_pivot = data.pivot_table(index=['purpose_category'], values='debt', aggfunc='mean')
data_pivot['debt'] = data_pivot['debt'] * 100
data_pivot['debt'] = data_pivot['debt'].round(2)
data_pivot.sort_values(by='debt')
# с помощью сводной таблицы получаем долю неплательщиков в разбивке по целям кредитования

,debt
purpose_category,
операции с недвижимостью,7.26
проведение свадьбы,7.91
получение образования,9.25
операции с автомобилем,9.35


**Вывод:** 
Возможно предположить, что кредит, выданный на операции с недвижимостью (доля неплательщиков - 7,26%), будет возвращен в срок с большей вероятностью, чем выданный на сделку с автомобилем (доля - 9,35%) или образование (доля - 9,25%). Возможно, ключевым фактором в данном случае является залог приобретенной на кредит недвижимости либо более тщательная процедура отбора заемщиков (учитывая более высокий средний чек). Относительно невелика доля неплательщиков также среди клиентов, получивших кредиты на проведение свадьбы (7,91%), что может быть связано как с небольшим средним размером данного кредита, так и с влиянием иных факторов (подобных клиентов существенно меньше, чем клиентов иных категорий; возможно, многие клиенты гасят кредит за счет средств, подаренных им на свадьбу). На относительно высокую долю неплательщиков среди получивших кредиты на образование может влиять отсутствие обеспечения, молодой возраст заемщиков, отсутствие стабильного и/или достаточного дохода у них. Вместе с тем, как было отмечено выше, выборка клиентов не сбалансирована по целям кредитования (так, например, ссуд на операции с недвижимостью больше половины, кредиты на проведение свадьбы составляют чуть больше 10%).

#### 3.5 Приведем возможные причины появления пропусков в исходных данных.

В ходе анализа исходных данных было выявлено 2 174 строки (около 10% от общего количества), в которых находились пропуски (NaN) в столбцах days_employed (общий трудовой стаж в днях) и total_income (ежемесячный доход) - в каждой из этих 2 174 строк отсутствовали данные в указанных двух столбцах.
При этом необходимо отметить, что распределение клиентов с пропусками по различным категориям (например, по типу занятости, цели кредитования или по возрасту) не свидетельствует о наличии какой-либо закономерности (данные были пропущены и по сотрудникам, и по пенсионерам, и по госслужащим, например). Кроме того, доля неплательщиков в данной выборке (8,5%) даже несколько ниже аналогичной доли в общей выборке всех клиентов. Таким образом, отсутствие в исходной базе соответствующих данных само по себе не свидетельствует о потенциальном/фактическом низком качестве ссуд клиентов (хотя показатель ежемесячного дохода по сути является одним из ключевых для оценки кредитоспособности).
Таким образом, возможно предположить, что данные о занятости и доходе по ряду клиентов не попали в базу данных из-за человеческого фактора или некоего технического сбоя при переносе сведений (в зависимости от способа наполнения базы первичными сведениями).

Также возможно предположить, что данные клиенты **после получения кредитов** по различным причинам (увольнение, человеческий фактор) не подтвердили документально свой ранее заявленный доход в ходе очередного периодического мониторинга, в связи с чем Банк исключил соответствующие сведения из базы. Однако, **против** данной гипотезы говорит факт отсутствия пропусков в столбце income_type. 

#### 3.6 Почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

Потому что медианное значение представляет собой некую усредненную величину - при этом его не искажают аномально низкие или высокие значения, как арифметическое среднее.

### Шаг 4: общий вывод.

В целях выявления взаимосвязи различных параметров заемщиков с уровнем их кредитоспособности для построения эффективной скоринговой модели был проведен анализ базы текущих клиентов Банка (21 331 строка после обработки).
В ходе исследования был использован параметр **debt** (имел ли заемщик задолженность по возврату кредитов).
Для формирования выводов было выдвинуто несколько гипотез:

- Существует зависимость между количеством детей и возвратом кредита в срок.
- Существует зависимость между семейным положением и возвратом кредита в срок.
- Существует зависимость между уровнем дохода и возвратом кредита в срок.
- Существует зависимость между целью кредита и его возвратом в срок.

По результатам анализа установлено следующее.

**Гипотеза 1 - подтверждена частично.**
Определенная зависимость между **фактом** наличия детей и возвратом кредита в срок имеется: доля имевших задолженность по кредиту среди бездетных **ниже** (доля неплательщиков среди них составляет 7,54%, доля неплательщиков среди имеющих детей составляет 9,24%).
При этом прямой зависимости между **количеством** детей и возвратом кредита в срок **не обнаружено** (выборка несимметрична, требуется дополнительный анализ на большем массиве).

Таким образом, возможно сделать вывод, что **более надежным** при прочих равных является заемщик **без детей**.

**Гипотеза 2 - подтверждена.**
Определенная зависимость между семейным положением и возвратом кредита в срок **выявлена**: доля имевших задолженность по кредиту среди не состоящих в браке (9,76%) и состоящих в гражданском браке (9,31%) **выше**, чем в других категориях (6,62% - 7,56%).

Таким образом, возможно сделать вывод, что **более надежным** при прочих равных является заемщик, **состоящий либо состоявший в браке** (при этом выборка по разведенным и овдовевшим **не репрезентативна**).

**Гипотеза 3 - подтверждена частично.**
Среди двух основных категорий клиентов по доходам (B и C) ожидаемо **меньше** доля неплательщиков среди **зарабатывающих больше** (7,06% против 8,5%). 
Клиентов, зарабатывающих менее 30 тыс. и более 1 млн руб., очень мало (0,2%), в связи с чем выборка по ним **не может быть признана репрезентативной в полной мере**. Доля неплательщиков в категории клиентов с доходом от 30 тыс. до 50 тыс. **ниже (6,02%)**, чем в других категориях, однако таких клиентов также **относительно немного** (1,6%).

В целом заемщики с **более высоким доходом** являются **более надежными**, однако целесообразно провести более детальный анализ на предмет определения влияния иных факторов на платежную дисциплину клиентов Банка из выборки и выявления более четких закономерностей.

**Гипотеза 4 - подтверждена.**
Кредит, выданный на **операции с недвижимостью** (доля неплательщиков - 7,26%), будет возвращен в срок с **большей вероятностью**, чем выданный на иные цели (9,25% - 9,35%). Относительно невелика доля неплательщиков также среди клиентов, получивших кредиты **на проведение свадьбы** (7,91%). Вместе с тем, выборка клиентов по целям кредитования **не сбалансирована** (ссуд на операции с недвижимостью больше половины, а кредиты на проведение свадьбы составляют чуть больше 10%).


**Общие замечания и рекомендации:**

1. Выборка клиентов **не вполне сбалансирована** по всем параметрам, использованным в ходе анализа (количество детей, семейное положение, уровень дохода, цель кредитования), в связи с чем при формировании окончательных выводов целесообразно учитывать возможное дополнительное влияние иных факторов.
Также следует обратить внимание на ряд допущений, принятых в ходе исследования - указанные ключевые параметры были статичны и актуальны на момент выдачи кредита **и** на момент выгрузки среза (мы **исключаем** их возможную динамику в течение какого-либо промежутка времени).

2. Для формирования максимально адекватных выводов по данной выборке клиентов в базе данных **не хватает** ключевых показателей - длительности просрочки и ее величины, характеризующих платежную дисциплину заемщика. Очевидно, что просрочка может быть обусловлена человеческим фактором или техническими причинами (может длиться 1 день и/или составлять 1 рубль, например). В данной связи столбец 'debt' (имел ли клиент задолженность по возврату кредитов) нуждается в дополнительной детализации, либо пояснениях (например, что клиенты с единицей в данном столбце - только "злостные" крупные неплательщики). Выводы, основанные лишь на факте просрочки, с высокой долей вероятности будут искажены и не объективны. Отказываясь от клиентов, аналогичных **всем** заемщикам, имевшим **любую** задолженность по кредиту, мы рискуем упустить возможности и дополнительную прибыль.